# RAG with Chat, Embed, and Rerank

This notebook shows how to build a RAG-powered chatbot with Cohere's Chat endpoint.  The chatbot can extract relevant information from external documents and produce verifiable, inline citations in its responses.

This application will use several Cohere API endpoints:

- Chat: For handling the main logic of the chatbot, including turning a user message into queries, generating responses, and producing citations
- Embed: For turning textual documents into their embeddings representation, later to be used in retrieval (we’ll use the latest, state-of-the-art Embed v3 model)
- Rerank: For reranking the retrieved documents according to their relevance to a query

The diagram below provides an overview of what we’ll build.


Here is a summary of the steps involved.

Initial phase:
- **Step 0**: Ingest the documents – get documents, chunk, embed, and index.

For each user-chatbot interaction:
- **Step 1**: Get the user message
- **Step 2**: Call the Chat endpoint in query-generation mode
- If at least one query is generated
  - **Step 3**: Retrieve and rerank relevant documents
  - **Step 4**: Call the Chat endpoint in document mode to generate a grounded response with citations
- If no query is generated
  - **Step 4**: Call the Chat endpoint in normal mode to generate a response

# Setup

In [1]:
! pip install cohere hnswlib unstructured -q

In [1]:
import cohere
import uuid
import hnswlib #vector library
from typing import List, Dict
from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title

co = cohere.Client("F8OTjXKgp4KZrk3UBga3jwrmg3opliFg7XqGvXhf") # Get your API key here: https://dashboard.cohere.com/api-keys

# Create a vector store for ingestion and retrieval


![RAG components - Vectorstore](../images/llmu/rag/rag-components-vectorstore.png)


First, we define the list of documents we want to ingest and make available for retrieval. As an example, we'll use the contents from the first module of Cohere's *LLM University: What are Large Language Models?*.

In [3]:
# raw_documents = [
#     {
#          "title": "Text Embeddings",
#         "url": "https://docs.cohere.com/docs/text-embeddings"},
#     {
#         "title": "Similarity Between Words and Sentences",
#         "url": "https://docs.cohere.com/docs/similarity-between-words-and-sentences"},
#     {
#         "title": "The Attention Mechanism",
#         "url": "https://docs.cohere.com/docs/the-attention-mechanism"},
#     {
#         "title": "Transformer Models",
#         "url": "https://docs.cohere.com/docs/transformer-models"}
# ]

In [2]:
raw_documents = [
    {
        "title": "Verdes by Haven",
        "url": "https://www.aldar.com/VerdesbyHaven/"},
    {
        "title": "Aldar Sustainability Strategy",
        "url": "https://www.aldar.com/en/explore-aldar/sustainability/strategy"},
    {
        "title": "The Properties",
        "url": "https://www.aldar.com/properties/en"},
    {
        "title": "Company Strategy",
        "url": "https://www.aldar.com/en/explore-aldar/about-aldar/strategy"}
]

In [3]:
class Vectorstore:
    """
    A class representing a collection of documents indexed into a vectorstore.

    Parameters:
    raw_documents (list): A list of dictionaries representing the sources of the raw documents. Each dictionary should have 'title' and 'url' keys.

    Attributes:
    raw_documents (list): A list of dictionaries representing the raw documents.
    docs (list): A list of dictionaries representing the chunked documents, with 'title', 'text', and 'url' keys.
    docs_embs (list): A list of the associated embeddings for the document chunks.
    docs_len (int): The number of document chunks in the collection.
    idx (hnswlib.Index): The index used for document retrieval.

    Methods:
    load_and_chunk(): Loads the data from the sources and partitions the HTML content into chunks.
    embed(): Embeds the document chunks using the Cohere API.
    index(): Indexes the document chunks for efficient retrieval.
    retrieve(): Retrieves document chunks based on the given query.
    """

    def __init__(self, raw_documents: List[Dict[str, str]]):
        self.raw_documents = raw_documents
        self.docs = []
        self.docs_embs = []
        self.retrieve_top_k = 10
        self.rerank_top_k = 3
        self.load_and_chunk()
        self.embed()
        self.index()


    def load_and_chunk(self) -> None:
        """
        Loads the text from the sources and chunks the HTML content.
        """
        print("Loading documents...")

        for raw_document in self.raw_documents:
            elements = partition_html(url=raw_document["url"])
            chunks = chunk_by_title(elements)
            for chunk in chunks:
                self.docs.append(
                    {
                        "title": raw_document["title"],
                        "text": str(chunk),
                        "url": raw_document["url"],
                    }
                )

    def embed(self) -> None:
        """
        Embeds the document chunks using the Cohere API.
        """
        print("Embedding document chunks...")

        batch_size = 90
        self.docs_len = len(self.docs)
        for i in range(0, self.docs_len, batch_size):
            batch = self.docs[i : min(i + batch_size, self.docs_len)]
            texts = [item["text"] for item in batch]
            docs_embs_batch = co.embed(
                texts=texts, model="embed-english-v3.0", input_type="search_document"
            ).embeddings
            self.docs_embs.extend(docs_embs_batch)

    def index(self) -> None:
        """
        Indexes the document chunks for efficient retrieval.
        """
        print("Indexing document chunks...")

        self.idx = hnswlib.Index(space="ip", dim=1024)
        self.idx.init_index(max_elements=self.docs_len, ef_construction=512, M=64)
        self.idx.add_items(self.docs_embs, list(range(len(self.docs_embs))))

        print(f"Indexing complete with {self.idx.get_current_count()} document chunks.")

    def retrieve(self, query: str) -> List[Dict[str, str]]:
        """
        Retrieves document chunks based on the given query.

        Parameters:
        query (str): The query to retrieve document chunks for.

        Returns:
        List[Dict[str, str]]: A list of dictionaries representing the retrieved document chunks, with 'title', 'text', and 'url' keys.
        """

        # Dense retrieval
        query_emb = co.embed(
            texts=[query], model="embed-english-v3.0", input_type="search_query"
        ).embeddings

        doc_ids = self.idx.knn_query(query_emb, k=self.retrieve_top_k)[0][0]

        # Reranking
        rank_fields = ["title", "text"] # We'll use the title and text fields for reranking

        docs_to_rerank = [self.docs[doc_id] for doc_id in doc_ids]

        rerank_results = co.rerank(
            query=query,
            documents=docs_to_rerank,
            top_n=self.rerank_top_k,
            model="rerank-english-v3.0",
           # rank_fields=rank_fields
        )

        doc_ids_reranked = [doc_ids[result.index] for result in rerank_results.results]

        docs_retrieved = []
        for doc_id in doc_ids_reranked:
            docs_retrieved.append(
                {
                    "title": self.docs[doc_id]["title"],
                    "text": self.docs[doc_id]["text"],
                    "url": self.docs[doc_id]["url"],
                }
            )

        return docs_retrieved

In the code cell below, we initialize an instance of the `Vectorstore` class and pass in the `raw_documents` list as input.

In [4]:
# Create an instance of the Vectorstore class with the given sources
vectorstore = Vectorstore(raw_documents)

Loading documents...
Embedding document chunks...
Indexing document chunks...
Indexing complete with 48 document chunks.


The `Vectorstore` class also has a `retrieve()` method, which we'll use to retrieve relevant document chunks given a query .  This method has two components: (1) dense retrieval, and (2) reranking.

### Dense retrieval

First, we embed the query using the same `embed-english-v3.0` model we used to embed the document chunks, but this time we set `input_type="search_query"`.

Search is performed by the `knn_query()` method from the `hnswlib` library. Given a query, it returns the document chunks most similar to the query. We can define the number of document chunks to return using the attribute `self.retrieve_top_k=10`.

### Reranking

After semantic search, we implement a reranking step.  While our semantic search component is already highly capable of retrieving relevant sources, the [Rerank endpoint](https://cohere.com/rerank) provides an additional boost to the quality of the search results, especially for complex and domain-specific queries. It takes the search results and sorts them according to their relevance to the query.

We call the Rerank endpoint with the `co.rerank()` method and define the number of top reranked document chunks to retrieve using the attribute `self.rerank_top_k=3`.  The model we use is `rerank-english-v2.0`.  

This method returns the top retrieved document chunks `chunks_retrieved` so that they can be passed to the chatbot.

In the code cell below, we check the document chunks that are retrieved for the query `"multi-head attention definition"`.

## Test Retrieval

In [16]:
vectorstore.retrieve("what is company strategy")

[{'title': 'Company Strategy',
  'text': "Our Company's\n\nStrategy\n\nOur Purpose\n\nWe build eco-systems around an intimate understanding of our stakeholders.\n\nOur work is well-designed for communities to thrive and raise their families, for corporate cultures to innovate & prosper and for schools where young talent meets great teaching.\n\nWe are human centric with a collection of talented people, businesses and suppliers coming together in a powerful eco-system built on strong relationships.",
  'url': 'https://www.aldar.com/en/explore-aldar/about-aldar/strategy'},
 {'title': 'Company Strategy',
  'text': 'Shaping the journey towards growth\n\nOur strategy shapes our journey towards future growth, enriching customer experiences and maximising shareholder value as we pursue our goals and realise our vision.\n\nOperational Excellence\n\nCustomer Centricity\n\nGrowth and Expansion\n\nPeople, Innovation and Digital Transformation\n\nOperational Excellence\n\nOur agile processes ensur

# Create a chatbot


Next, we implement a class to handle the interaction between the user and the chatbot.  It takes an instance of the `Vectorstore` class as input.

The `run()` method will be used to run the chatbot application.  It begins with the logic for getting the user message, along with a way for the user to end the conversation.  

Based on the user message, the chatbot needs to decide if it needs to consult external information before responding.  If so, the chatbot determines an optimal set of search queries to use for retrieval.  When we call `co.chat()` with `search_queries_only=True`, the Chat endpoint handles this for us automatically.

The generated queries can be accessed from the `search_queries` field of the object that is returned.  Then, what happens next depends on how many queries are returned.
- If queries are returned, we call the `retrieve()` method of the Vectorstore object for the  retrieval step.  The retrieved document chunks are then passed to the Chat endpoint by adding a `documents` parameter when we call `co.chat()` again.
- Otherwise, if no queries are returned, we call the Chat endpoint another time, passing the user message and without needing to add any documents to the call.

In either case, we also pass the `conversation_id` parameter, which retains the interactions between the user and the chatbot in the same conversation thread. We also enable the `stream` parameter so we can stream the chatbot response.

We then print the chatbot's response.  In the case that the external information was used to generate a response, we also display citations.

In [17]:
class Chatbot:
    def __init__(self, vectorstore: Vectorstore):
        """
        Initializes an instance of the Chatbot class.

        Parameters:
        vectorstore (Vectorstore): An instance of the Vectorstore class.

        """
        self.vectorstore = vectorstore
        self.conversation_id = str(uuid.uuid4())

    def run(self):
        """
        Runs the chatbot application.

        """
        while True:
            print(f"\n{'-'*100}\n")
            # Get the user message
            message = input("User: ")

            # Typing "quit" ends the conversation
            if message.lower() == "quit":
              print("Ending chat.")
              break
           

            # Generate search queries (if any)
            response = co.chat(message=message,
                               model="command-r",
                               search_queries_only=True)

            # If there are search queries, retrieve document chunks and respond
            if response.search_queries:
                print("Retrieving information...", end="")

                # Retrieve document chunks for each query
                documents = []
                for query in response.search_queries:
                    documents.extend(self.vectorstore.retrieve(query.text))

                # Use document chunks to respond
                response = co.chat_stream(
                    message=message,
                    model="command-r-plus",
                    documents=documents,
                    conversation_id=self.conversation_id,
                )

            # If there is no search query, directly respond
            else:
                response = co.chat_stream(
                    message=message,
                    model="command-r-plus",
                    conversation_id=self.conversation_id,
                )

            # Print the chatbot response, citations, and documents
            print("\nChatbot:")
            citations = []
            cited_documents = []

            # Display response
            for event in response:
                if event.event_type == "text-generation":
                    print(event.text, end="")
                elif event.event_type == "citation-generation":
                    citations.extend(event.citations)
                elif event.event_type == "stream-end":
                    cited_documents = event.response.documents

            # Display citations and source documents
            if citations:
              print("\n\nCITATIONS:")
              for citation in citations:
                print(citation)

              print("\nDOCUMENTS:")
              for document in cited_documents:
                print(document)

# Run the chatbot

We can now run the chatbot.  For this, we create the instance of `Chatbot` and run the chatbot by invoking the `run()` method.

The format of each citation is:
- `start`: The starting point of a span where one or more documents are referenced
- `end`: The ending point of a span where one or more documents are referenced
- `text`: The text representing this span
- `document_ids`: The IDs of the documents being referenced (`doc_0` being the ID of the first document passed to the `documents` creating parameter in the endpoint call, and so on)

In [18]:
# Create an instance of the Chatbot class
chatbot = Chatbot(vectorstore)

# Run the chatbot
chatbot.run()


----------------------------------------------------------------------------------------------------

User: what is the company strategy
Retrieving information...
Chatbot:
Aldar's company strategy is to shape the journey towards growth, enriching customer experiences and maximising shareholder value. The company's strategic objectives are:
- Future-proofing fiscal strength
- Driving maximum financial returns for shareholders by continually growing its portfolio and exploring new opportunities
- Operational excellence
- Customer centricity
- Growth and expansion
- People, innovation and digital transformation

Aldar's purpose is to build ecosystems around an intimate understanding of its stakeholders. The company's work is designed for communities to thrive, corporate cultures to innovate and prosper, and for schools to foster young talent. Aldar is human-centric, bringing together talented people, businesses, and suppliers in a powerful ecosystem built on strong relationships.

CITATI

User: What are the properties project Aldar has
Retrieving information...
Chatbot:
Sorry, I was unable to find any information about Aldar's properties projects.
----------------------------------------------------------------------------------------------------



KeyboardInterrupt: Interrupted by user